# CrewAI: Orchestrating Collaborative Agent Systems

**Objective:** This notebook provides a hands-on guide to CrewAI, a framework designed for engineering sophisticated multi-agent systems. You will learn how to create a "crew" of autonomous AI agents that collaborate to achieve complex goals.

**Target Audience:** Software engineers attending the AI-Driven Software Engineering Program.

**Core Philosophy:** CrewAI's strength lies in its intuitive, high-level abstractions for defining agents with specific roles, assigning them tasks, and defining the workflow (process) they should follow. It simplifies the creation of systems where multiple agents work together, each contributing its specialized skills.

## 1. Setup

First, we'll install the necessary libraries. CrewAI uses LangChain components under the hood. We will also install the `tavily-python` library directly to create a custom search tool, which helps avoid complex dependency issues.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY") or not os.getenv("TAVILY_API_KEY"):
    print("ERROR: OPENAI_API_KEY or TAVILY_API_KEY not found. Please check your .env file.")

## 2. Foundational Crew: The Two-Agent Team

Our first example will be a simple, two-agent crew designed to plan a trip. This will introduce the three core concepts of CrewAI:

-   **`Agent`**: A role-playing entity with a specific `role`, `goal`, and `backstory`.
-   **`Task`**: A specific unit of work to be performed by an agent.
-   **`Crew`**: A collection of agents and tasks, along with a defined `process` for execution.

In [2]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

# Use a LangChain model as the LLM for the agents
llm = ChatOpenAI(model="gpt-4o")

# Agent 1: The Travel Expert
travel_agent = Agent(
    role='Expert Travel Agent',
    goal='Create a detailed, 3-day itinerary for a trip.',
    backstory='You have 20 years of experience in luxury travel planning and know all the hidden gems.',
    verbose=True,
    llm=llm
)

# Agent 2: The Local Food Critic
food_critic = Agent(
    role='Local Food Critic',
    goal='Recommend the best, most authentic local restaurants for the trip.',
    backstory='You are a famous food blogger who lives in the destination city and is an expert on its culinary scene.',
    verbose=True,
    llm=llm
)

# Task 1: Plan the Itinerary (for the Travel Agent)
plan_itinerary = Task(
    description='Create a 3-day travel itinerary for a trip to Tokyo, Japan. Focus on cultural sites and activities.',
    expected_output='A markdown file with a day-by-day plan, including morning, afternoon, and evening activities.',
    agent=travel_agent
)

# Task 2: Recommend Restaurants (for the Food Critic)
# This task uses the output of the first task as its context.
recommend_restaurants = Task(
    description='Based on the planned itinerary, recommend one authentic restaurant for each day of the trip.',
    expected_output='A list of 3 restaurant names, each with a brief description and why it fits the itinerary.',
    agent=food_critic,
    context=[plan_itinerary]
)

# Form the crew with a sequential process
trip_crew = Crew(
    agents=[travel_agent, food_critic],
    tasks=[plan_itinerary, recommend_restaurants],
    process=Process.sequential,
    verbose=True
)

print("--- Kicking off the Trip Planning Crew ---")
trip_result = trip_crew.kickoff()

print("\n--- Final Itinerary ---")
print(trip_result)

--- Kicking off the Trip Planning Crew ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 19855e57-264c-40c6-a4c1-0af6b5bacbd7                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Travel Agent                                                                                     │
│                                                                                                                 │
│  Task: Create a 3-day travel itinerary for a trip to Tokyo, Japan. Focus on cultural sites and activities.      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Travel Agent                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # 3-Day Tokyo Cultural Itinerary                                                                               │
│                                                                                                                 │
│  ## Day 1: Explore Traditional Tokyo                                                                            │
│                                                                                                                 │
│  ### Morning                                                                                                    │
│  - **Asakusa and Senso-ji Temple**                                                                              │
│    - Start your day with a visit to Asakusa, one of Tokyo’s oldest neighborhoods. Explore Senso-ji, the city's  │
│  oldest temple, renowned for its iconic Kaminarimon Gate and vibrant shopping street, Nakamise-dori.            │
│    - **Breakfast Idea:** Try traditional Japanese sweets like Ningyo-yaki (small cakes filled with red bean     │
│  paste) from one of the local stalls.                                                                           │
│                                                                                                                 │
│  ### Afternoon                                                                                                  │
│  - **Sumida River Cruise**                                                                                      │
│    - Enjoy a scenic boat ride on the Sumida River, providing views of the Tokyo Skytree and traditional         │
│  neighborhoods.                                                                                                 │
│  - **Edo-Tokyo Museum**                                                                                         │
│    - Head to the Edo-Tokyo Museum to learn about Tokyo’s transition from a small fishing village to a bustling  │
│  metropolis.                                                                                                    │
│                                                                                                                 │
│  ### Evening                                                                                                    │
│  - **Dinner in Ryogoku**                                                                                        │
│    - Dine at a chanko-nabe restaurant, a hearty hot pot meal traditionally eaten by sumo wrestlers, in the      │
│  sumo district of Ryogoku.                                                                                      │
│  - **Tokyo Skytree Observation Deck**                                                                           │
│    - Conclude your day with breathtaking views of the city from the Skytree's observation deck, especially      │
│  captivating at night.                                                                                          │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5182b20b-88ed-4dc4-9c13-20b4c13e29ca                                                                     │
│  Agent: Expert Travel Agent                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Task: Based on the planned itinerary, recommend one authentic restaurant for each day of the trip.             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **Day 1: Explore Traditional Tokyo**                                                                        │
│     - **Restaurant: Chanko Tomoegata**                                                                          │
│       - **Description:** Situated in Ryogoku, close to the sumo stables, Chanko Tomoegata offers an authentic   │
│  chanko-nabe experience. The restaurant is steeped in sumo culture, with memorabilia adorning its walls. It's   │
│  the perfect place to immerse yourself in the atmosphere and flavors of this heartwarming dish, traditionally   │
│  enjoyed by sumo wrestlers.                                                                                     │
│       - **Why it fits the itinerary:** After a day exploring traditional Tokyo and the Edo-Tokyo Museum,        │
│  dining in the sumo district enhances the cultural experience, tying together the themes of tradition and       │
│  history.                                                                                                       │
│                                                                                                                 │
│  2. **Day 2: Dive into Tokyo’s Cultural Arts**                                                                  │
│     - **Restaurant: Ginza Toyoda**                                                                              │
│       - **Description:** Ginza Toyoda, located near the Kabuki-za Theatre, serves exquisite kaiseki, a          │
│  traditional multi-course Japanese dinner that is as much about presentation and experience as it is about      │
│  taste. Each dish is crafted to reflect the season and is aesthetically arranged, symbolizing the artistic      │
│  focus of your day.                                                                                             │
│       - **Why it fits the itinerary:** After a day immersed in arts and a potential kabuki performance, this    │
│  dinner offers a continuation of cultural appreciation through culinary artistry in the heart of Ginza.         │
│                                                                                                                 │
│  3. **Day 3: Embrace Tokyo’s Serenity and Zen**                                                                 │
│     - **Restaurant: Kappo Nakajima**                                                                            │
│       - **Description:** Tucked away in Shinjuku, Kappo Nakajima specializes in the humble yet revered          │
│  sardine, or "iwashi," prepared in a variety of traditional methods. This Michelin-starred restaurant offers    │
│  an intimate dining experience with its focus on seasonal and simple flavors.                                   │
│       - **Why it fits the itinerary:** After a serene day at Meiji Shrine and the Nezu Museum, finishing with   │
│  dinner here provides a fitting end with a focus on traditional Japanese cooking techniques. Continue the       │
│  experience in Shinjuku's atmospheric Golden Gai for a feel of Tokyo’s vibrant nightlife.                       │
│                                                                                                                 │
│  Enjoy your cultural and culinary journey through Tokyo!                                                        │
│                                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9b9e9722-9c1d-4f60-9e1a-64181c2957d9                                                                     │
│  Agent: Local Food Critic                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 19855e57-264c-40c6-a4c1-0af6b5bacbd7                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. **Day 1: Explore Traditional Tokyo**                                                          │
│     - **Restaurant: Chanko Tomoegata**                                                                          │
│       - **Description:** Situated in Ryogoku, close to the sumo stables, Chanko Tomoegata offers an authentic   │
│  chanko-nabe experience. The restaurant is steeped in sumo culture, with memorabilia adorning its walls. It's   │
│  the perfect place to immerse yourself in the atmosphere and flavors of this heartwarming dish, traditionally   │
│  enjoyed by sumo wrestlers.                                                                                     │
│       - **Why it fits the itinerary:** After a day exploring traditional Tokyo and the Edo-Tokyo Museum,        │
│  dining in the sumo district enhances the cultural experience, tying together the themes of tradition and       │
│  history.                                                                                                       │
│                                                                                                                 │
│  2. **Day 2: Dive into Tokyo’s Cultural Arts**                                                                  │
│     - **Restaurant: Ginza Toyoda**                                                                              │
│       - **Description:** Ginza Toyoda, located near the Kabuki-za Theatre, serves exquisite kaiseki, a          │
│  traditional multi-course Japanese dinner that is as much about presentation and experience as it is about      │
│  taste. Each dish is crafted to reflect the season and is aesthetically arranged, symbolizing the artistic      │
│  focus of your day.                                                                                             │
│       - **Why it fits the itinerary:** After a day immersed in arts and a potential kabuki performance, this    │
│  dinner offers a continuation of cultural appreciation through culinary artistry in the heart of Ginza.         │
│                                                                                                                 │
│  3. **Day 3: Embrace Tokyo’s Serenity and Zen**                                                                 │
│     - **Restaurant: Kappo Nakajima**                                                                            │
│       - **Description:** Tucked away in Shinjuku, Kappo Nakajima specializes in the humble yet revered          │
│  sardine, or "iwashi," prepared in a variety of traditional methods. This Michelin-starred restaurant offers    │
│  an intimate dining experience with its focus on seasonal and simple flavors.                                   │
│       - **Why it fits the itinerary:** After a serene day at Meiji Shrine and the Nezu Museum, finishing with   │
│  dinner here provides a fitting end with a focus on traditional Japanese cooking techniques. Continue the       │
│  experience in Shinjuku's atmospheric Golden Gai for a feel of Tokyo’s vibrant nightlife.                       │
│                                                                                                                 │
│  Enjoy your cultural and culinary journey through Toky


--- Final Itinerary ---
1. **Day 1: Explore Traditional Tokyo**
   - **Restaurant: Chanko Tomoegata**
     - **Description:** Situated in Ryogoku, close to the sumo stables, Chanko Tomoegata offers an authentic chanko-nabe experience. The restaurant is steeped in sumo culture, with memorabilia adorning its walls. It's the perfect place to immerse yourself in the atmosphere and flavors of this heartwarming dish, traditionally enjoyed by sumo wrestlers.
     - **Why it fits the itinerary:** After a day exploring traditional Tokyo and the Edo-Tokyo Museum, dining in the sumo district enhances the cultural experience, tying together the themes of tradition and history.

2. **Day 2: Dive into Tokyo’s Cultural Arts**
   - **Restaurant: Ginza Toyoda**
     - **Description:** Ginza Toyoda, located near the Kabuki-za Theatre, serves exquisite kaiseki, a traditional multi-course Japanese dinner that is as much about presentation and experience as it is about taste. Each dish is crafted to refle

## 3. Adding Tools to a CrewAI Agent

CrewAI agents can use tools to access external information and capabilities. Instead of relying on pre-built wrappers which can cause dependency issues, we can create our own custom tool by wrapping the Tavily Python client. This is a robust and flexible approach.

In [3]:
from crewai.tools import BaseTool
from tavily import TavilyClient

# 1. Create a custom tool by inheriting from BaseTool
class TavilySearchTool(BaseTool):
    name: str = "TavilySearch"
    description: str = "A tool that can be used to search the web with Tavily for up-to-date information."
    
    def _run(self, query: str) -> str:
        client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
        response = client.search(query=query, search_depth="basic")
        return response['results']

# Instantiate our custom tool
search_tool = TavilySearchTool()

# 2. Create an agent and assign the custom tool to it
market_researcher = Agent(
    role='Market Research Analyst',
    goal='Find and summarize the latest news about a specific company.',
    backstory='You are a skilled analyst who can quickly find and synthesize financial news.',
    tools=[search_tool], # Assign the custom tool here
    verbose=True,
    llm=llm
)

# 3. Create a task for the agent
research_company_task = Task(
    description='Find the top 3 news headlines for NVIDIA (stock ticker NVDA) from the last 24 hours and summarize them.',
    expected_output='A numbered list of 3 headlines, each followed by a one-sentence summary.',
    agent=market_researcher
)

# 4. Form a single-agent crew to run the task
research_crew = Crew(
    agents=[market_researcher],
    tasks=[research_company_task],
    verbose=True
)

print("--- Kicking off the Research Crew with Tools ---")
research_result = research_crew.kickoff()

print("\n--- Final Research Summary ---")
print(research_result)

--- Kicking off the Research Crew with Tools ---


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a587641f-debb-43ea-ab18-0101744bf161                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Find the top 3 news headlines for NVIDIA (stock ticker NVDA) from the last 24 hours and summarize them.  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/agaleana/repos/AG-AISOFTDEV/.venv/lib/python3.11/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Thought: I need to search for the latest news about NVIDIA from the last 24 hours to identify the     │
│  top 3 headlines and summarize them.                                                                            │
│                                                                                                                 │
│  Using Tool: TavilySearch                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [{'url': 'https://nvidianews.nvidia.com/news/latest', 'title': 'Latest News | NVIDIA Newsroom', 'content':     │
│  'Latest News · Industry Leaders Transform Enterprise Data Centers for the AI Era With NVIDIA RTX PRO Servers   │
│  · Take It for a Spin: NVIDIA Rolls Out DRIVE AGX Thor', 'score': 0.6155738, 'raw_content': None}, {'url':      │
│  'https://www.cnbc.com/quotes/NVDA', 'title': 'NVDA: NVIDIA Corp - Stock Price, Quote and News', 'content':     │
│  'NVIDIA Corp NVDA:NASDAQ ; after hours icon After Hours: Last | 9:25 AM EDT. 182.27 quote price arrow up       │
│  +0.50 (+0.28%) ; Volume. 5,462,961.', 'score': 0.5752307, 'raw_content': None}, {'url':                        │
│  'https://ca.finance.yahoo.com/quote/NVDA/news/', 'title': 'NVIDIA Corporation (NVDA) Latest Stock News &       │
│  Headlines', 'content': 'Get the latest NVIDIA Corporation (NVDA) stock news and headlines to help you in your  │
│  trading and investing decisions.', 'score': 0.49538648, 'raw_content': None}, {'url':                          │
│  'https://www.cnn.com/markets/stocks/NVDA', 'title': 'NVDA Stock Quote Price and Forecast', 'content': 'Latest  │
│  NVDA news ; 5:06am ET ChatGPT-Maker OpenAI Employee Stock Sale Could Hit $8B ; 8:47pm ET OpenAI to Fix         │
│  ChatGPT for “Sensitive Situations” After Teen', 'score': 0.4680374, 'raw_content': None}, {'url':              │
│  'https://www.cnbc.com/2025/08/26/stock-market-today-live-updates.html', 'title': 'S&P 500 inches higher as     │
│  traders brace for Nvidia earnings', 'content': "Wall Street has high hopes that Nvidia's financial results,    │
│  expected after the closing bell on Wednesday, can reignite the tech rally.", 'score': 0.39796236,              │
│  'raw_content': None}]                                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. Title: Industry Leaders Transform Enterprise Data Centers for the AI Era With NVIDIA RTX PRO Servers        │
│     Summary: NVIDIA's latest RTX PRO servers are being adopted by leading industries to modernize enterprise    │
│  data centers, marking a significant step towards the AI era.                                                   │
│                                                                                                                 │
│  2. Title: Take It for a Spin: NVIDIA Rolls Out DRIVE AGX Thor                                                  │
│     Summary: NVIDIA has introduced the DRIVE AGX Thor, an advanced platform aimed at enhancing automotive       │
│  tech, highlighting a push towards innovation in autonomous driving solutions.                                  │
│                                                                                                                 │
│  3. Title: Wall Street Has High Hopes for NVIDIA's Financial Results                                            │
│     Summary: Investors are optimistic about NVIDIA's upcoming financial results, hoping they will fuel a        │
│  resurgence in the overall tech market.                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6dd2e8ee-9eb2-4a88-9981-acb131d36aca                                                                     │
│  Agent: Market Research Analyst                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a587641f-debb-43ea-ab18-0101744bf161                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. Title: Industry Leaders Transform Enterprise Data Centers for the AI Era With NVIDIA RTX PRO  │
│  Servers                                                                                                        │
│     Summary: NVIDIA's latest RTX PRO servers are being adopted by leading industries to modernize enterprise    │
│  data centers, marking a significant step towards the AI era.                                                   │
│                                                                                                                 │
│  2. Title: Take It for a Spin: NVIDIA Rolls Out DRIVE AGX Thor                                                  │
│     Summary: NVIDIA has introduced the DRIVE AGX Thor, an advanced platform aimed at enhancing automotive       │
│  tech, highlighting a push towards innovation in autonomous driving solutions.                                  │
│                                                                                                                 │
│  3. Title: Wall Street Has High Hopes for NVIDIA's Financial Results                                            │
│     Summary: Investors are optimistic about NVIDIA's upcoming financial results, hoping they will fuel a        │
│  resurgence in the overall tech market.                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Final Research Summary ---
1. Title: Industry Leaders Transform Enterprise Data Centers for the AI Era With NVIDIA RTX PRO Servers
   Summary: NVIDIA's latest RTX PRO servers are being adopted by leading industries to modernize enterprise data centers, marking a significant step towards the AI era.
  
2. Title: Take It for a Spin: NVIDIA Rolls Out DRIVE AGX Thor
   Summary: NVIDIA has introduced the DRIVE AGX Thor, an advanced platform aimed at enhancing automotive tech, highlighting a push towards innovation in autonomous driving solutions.

3. Title: Wall Street Has High Hopes for NVIDIA's Financial Results
   Summary: Investors are optimistic about NVIDIA's upcoming financial results, hoping they will fuel a resurgence in the overall tech market.


## 4. Advanced Capability: Hierarchical Process

For more complex workflows, CrewAI offers a `hierarchical` process. In this mode, a manager agent is dynamically nominated to orchestrate the crew, delegating tasks to other agents and synthesizing the final result. This is suitable for problems that require more dynamic coordination.

In [4]:
# Ensure Task, Crew, and Process are available (in case earlier cells haven't been run)
from crewai import Task, Crew, Process

# Re-using the agents from our first example

# The tasks are the same, but we don't need to specify context
plan_itinerary_h = Task(
    description='Create a 3-day travel itinerary for a trip to Rome, Italy. Focus on historical sites.',
    expected_output='A markdown file with a day-by-day plan.',
    agent=travel_agent
)

recommend_restaurants_h = Task(
    description='Recommend one authentic restaurant for each day of the trip to Rome.',
    expected_output='A list of 3 restaurant names with descriptions.',
    agent=food_critic
)

# Form the crew with a hierarchical process
hierarchical_crew = Crew(
    agents=[travel_agent, food_critic],
    tasks=[plan_itinerary_h, recommend_restaurants_h],
    process=Process.hierarchical,  # Use the hierarchical process
    manager_llm=llm # Specify an LLM for the manager agent
)

print("--- Kicking off the Hierarchical Crew ---")
hierarchical_result = hierarchical_crew.kickoff()

print("\n--- Final Hierarchical Result ---")
print(hierarchical_result)

--- Kicking off the Hierarchical Crew ---


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Travel Agent                                                                                     │
│                                                                                                                 │
│  Task: What are the top historical sites to visit in Rome, Italy, for a 3-day itinerary focusing on history?    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert Travel Agent                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Your 3-day itinerary for exploring historical sites in Rome, Italy, ensuring a comprehensive experience of     │
│  its rich history, is as follows:                                                                               │
│                                                                                                                 │
│  **Day 1: Ancient Rome Exploration**                                                                            │
│  - **Morning: Colosseum and Roman Forum**                                                                       │
│    - Start your day at the iconic Colosseum. Opt for a guided tour to delve into the history of ancient         │
│  gladiatorial games and architectural marvels.                                                                  │
│    - Next, walk through the Roman Forum, which was the center of political and social life in ancient Rome.     │
│  Don't miss the Temple of Saturn and Arch of Septimius Severus.                                                 │
│                                                                                                                 │
│  - **Afternoon: Palatine Hill**                                                                                 │
│    - Continue to Palatine Hill, the most famous of Rome's seven hills. Enjoy the panoramic views of the ruins   │
│  and explore the remnants of the imperial palaces.                                                              │
│                                                                                                                 │
│  - **Evening: Capitoline Hill and Museums**                                                                     │
│    - Visit Capitoline Hill and its spectacular piazza designed by Michelangelo. Explore the Capitoline          │
│  Museums, home to a vast collection of ancient Roman statues, inscriptions, and artifacts.                      │
│                                                                                                                 │
│  **Day 2: Renaissance and Baroque Rome**                                                                        │
│  - **Morning: The Pantheon and Piazza Navona**                                                                  │
│    - Begin your day at the Pantheon, marveling at one of the best-preserved ancient Roman buildings and a       │
│  symbol of architectural genius.                                                                                │
│    - Stroll to Piazza Navona, with its stunning Baroque fountains, including the Fountain of the Four Rivers    │
│  by Bernini.                                                                                                    │
│                                                                                                                 │
│  - **Afternoon: Vatican City**                                                                                  │
│    - Spend your afternoon in Vatican City. Tour St. Peter's Basilica and admire the breathtaking works of art,  │
│  such as Michelangelo's Pietà and Bernini's Baldachin.                                                          │
│    - Visit the Vatican Museums and the Sistine Chapel. To fully appreciate the art, consider booking a guided   │
│  tour focusing on the historical and religious signific

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Task: Can you recommend an authentic restaurant for Day 1 of the travel itinerary to Rome? This day focuses    │
│  on Ancient Rome Exploration, including visits to the Colosseum, Roman Forum, Palatine Hill, Capitoline Hill,   │
│  and Museums. We need a restaurant that complements this historical theme.                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  For an authentic dining experience that complements the historical theme of your Ancient Rome exploration, I   │
│  highly recommend "Ristorante Aroma at Palazzo Manfredi." This elegant restaurant offers stunning views of the  │
│  Colosseum, providing a breathtaking backdrop to your dining experience. The restaurant is renowned for its     │
│  excellent Italian cuisine that reflects traditional Roman flavors while using high-quality local ingredients.  │
│  Chef Giuseppe Di Iorio crafts a menu that highlights the culinary heritage of Rome with a modern touch,        │
│  making it an ideal place to immerse yourself in the city's historic ambiance. Enjoy a memorable meal amid the  │
│  sights and sounds of Ancient Rome, just moments away from the iconic monuments you've visited throughout the   │
│  day.                                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Task: Can you recommend an authentic restaurant for Day 2 of the travel itinerary to Rome? This day focuses    │
│  on Renaissance and Baroque Rome, including visits to the Pantheon, Piazza Navona, Vatican City, and Castel     │
│  Sant'Angelo.                                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  For an authentic dining experience on Day 2 of your Rome itinerary, which focuses on Renaissance and Baroque   │
│  Rome, I recommend dining at "Ristorante Arlu." Situated just a stone's throw away from Vatican City, this      │
│  family-run restaurant offers a charming ambiance that perfectly complements your day's sightseeing of          │
│  artistic and historical grandeur. Arlu is renowned for its traditional Roman cuisine, with dishes such as      │
│  Cacio e Pepe and Saltimbocca alla Romana prepared to perfection. Their pasta Fettuccine alla Papalina, a       │
│  classic Roman specialty, mirrors the elegance and artistry you'll see in places like St. Peter's Basilica.     │
│  With a cozy atmosphere and a commitment to local, high-quality ingredients, Ristorante Arlu provides a truly   │
│  immersive culinary experience that harmonizes beautifully with the Renaissance and Baroque masterpieces you    │
│  encounter throughout the day. Remember to make a reservation in advance to ensure a memorable meal after a     │
│  day of exploring these iconic sites.                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Task: Can you recommend an authentic restaurant for Day 3 of the travel itinerary to Rome? This day focuses    │
│  on hidden historical gems and local culture, including visits to the Baths of Caracalla, the Appian Way and    │
│  Catacombs, and the Trastevere district.                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Food Critic                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Given the Day 3 itinerary which features historical and cultural exploration in Rome, I highly recommend       │
│  dining at "Da Enzo al 29" in the Trastevere district. This quaint, authentic Roman trattoria is beloved by     │
│  locals and offers a true taste of traditional Roman cuisine. Known for its classic dishes like Carbonara and   │
│  Saltimbocca, Da Enzo al 29 uses fresh, local ingredients, and the rustic atmosphere adds to an authentic       │
│  Italian dining experience. Be sure to try their signature dessert, the Tiramisu, which has garnered rave       │
│  reviews for its perfect balance of coffee, cream, and cocoa. Remember that the restaurant is popular and       │
│  doesn't take reservations, so arriving early to secure a table in the evening would be a great strategy. This  │
│  choice immerses you fully in the local dining culture, perfectly complementing the day's exploration of        │
│  Rome's rich history and vibrant traditions.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


--- Final Hierarchical Result ---
1. **Ristorante Aroma at Palazzo Manfredi** - For Day 1, focused on Ancient Rome Exploration, Ristorante Aroma offers stunning views of the Colosseum, providing a breathtaking backdrop to your dining experience. The restaurant is known for its excellent Italian cuisine, reflecting traditional Roman flavors with high-quality local ingredients. Chef Giuseppe Di Iorio crafts a menu highlighting Rome's culinary heritage with a modern touch.

2. **Ristorante Arlu** - For Day 2, which focuses on Renaissance and Baroque Rome, Ristorante Arlu offers a charming ambiance near Vatican City. Known for its traditional Roman cuisine, featuring classic dishes like Cacio e Pepe and Saltimbocca alla Romana, this family-run restaurant is a perfect spot to indulge in the elegance and artistry of the day’s discoveries. Reservations are recommended.

3. **Da Enzo al 29** - For Day 3, which highlights hidden historical gems and local culture, Da Enzo al 29 in Trastevere of

## Lab Conclusion

In this lab, you've learned the fundamentals of CrewAI. You've seen how to define agents with distinct roles, create tasks for them to perform, and assemble them into a crew that can work together sequentially or be managed hierarchically.

**Key Takeaways:**
- CrewAI is excellent for problems that can be broken down into distinct roles and responsibilities.
- The `Agent`, `Task`, and `Crew` abstractions provide a clear and intuitive way to structure multi-agent systems.
- Creating custom tools by inheriting from `BaseTool` is a reliable way to add capabilities and avoid dependency conflicts.
- The `hierarchical` process allows for more dynamic, manager-led coordination for complex, non-linear tasks.